In [17]:
import numpy as np
import sys, os, time
import torch
import torch.nn as nn
import data, architecture
import optuna

################################### INPUT ############################################
# data parameters
f_Pk_norm = None #file with Pk to normalize Pk
seed      = 1         #seed to split data in train/valid/test
mode      = 'train'   #'train','valid','test' or 'all'

cosm_type = 'nwLH'
Pk_type = 'MPk'
name = 'transfer5_network2_w_log_test_final_layer_BSQ'
additional_extension = ''
log = False

f_Pk      = 'all_'+str(Pk_type)+'_'+str(cosm_type)+'_w_log_test_final_layer_fortransfer.npy'
# f_Pk = 'all_'+str(Pk_type)+'_'+str(cosm_type)+'.npy'
f_params  = '../real_params/'+str(cosm_type)+'_params.txt'



study_name = str(Pk_type)+'_'+str(cosm_type)+'_params_'+str(name)
fout = 'results/Results_'+mode+'_'+str(Pk_type)+'_'+str(cosm_type)+'_params_'+str(name)+str(additional_extension)+'.txt'
mother = '/scratch/network/vk9342/USRP2024_scratch/pytorch/'+str(Pk_type)+'_'+str(cosm_type)+'/'+str(name)+'/models/'

output_size = 6
input_size  = 5   #dimensions of input data
    
# architecture parameters
final_hidden_layer_size = 5
# training parameters
batch_size = 32
# optuna parameters
storage    = 'sqlite:///nwLH.db'

######################################################################################

# use GPUs if available
if torch.cuda.is_available():
    print("CUDA Available")
    device = torch.device('cuda')
else:
    print('CUDA Not Available')
    device = torch.device('cpu')

# load the optuna study
study = optuna.load_study(study_name=study_name, storage=storage)

# get the scores of the study trials
values = np.zeros(len(study.trials))
completed = 0
for i,t in enumerate(study.trials):
    values[i] = t.value
    if t.value is not None:  completed += 1

# get the info of the best trial
indexes = np.argsort(values)
for i in [0]:  #choose the best-model here, e.g. [0], or [1]
    trial = study.trials[indexes[i]]
    print("\nTrial number {}".format(trial.number))
    print("Value: %.5e"%trial.value)
    print(" Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    lr       = trial.params['lr']
    wd       = trial.params['wd']
    n_layers       = trial.params['n_layers']
    'comment h1 and dr out for dynamic models'
    # h1       = trial.params['h1']
    # dr       = trial.params['dr']
    fmodel = mother +'model_%d.pt'%trial.number

# # generate the architecture
model = architecture.dynamic_model(trial, input_size, output_size, #final_hidden_layer_size,
                                   n_layers,
                                   max_neurons_layers=500)


model.to(device)  

# load best-model, if it exists
if os.path.exists(fmodel):  
    print('Loading model...')
    model.load_state_dict(torch.load(fmodel, map_location=torch.device(device)))
else:
    raise Exception('model doesnt exists!!!')


# define loss function
criterion = nn.MSELoss() 

# get the data
test_loader = data.create_dataset(mode, seed, f_Pk, f_Pk_norm, f_params, 
                                  batch_size, shuffle=False, workers=1, cosm_type =cosm_type, log=log)
test_points = 0
for x,y in test_loader:  test_points += x.shape[0]

# define the arrays containing the true and predicted value of the parameters
params  = output_size
results = np.zeros((test_points, 2*params), dtype=np.float32)

# test the model
test_loss, points = 0.0, 0
model.eval()
with torch.no_grad():
    for x, y in test_loader:
        bs   = x.shape[0]  #batch size
        x, y = x.to(device), y.to(device)
        y_NN = model(x)
        test_loss += (criterion(y_NN, y).item())*bs
        results[points:points+bs,0*params:1*params] = y.cpu().numpy()
        results[points:points+bs,1*params:2*params] = y_NN.cpu().numpy()
        points    += bs
test_loss /= points
print('Test loss:', test_loss)

# denormalize results here
#
#

# save results to file
np.savetxt(fout, results)



CUDA Available

Trial number 37
Value: 1.33658e-02
 Params: 
    lr: 0.0005999994245809
    wd: 0.0015148848047548944
    n_layers: 3
    n_units_l0: 460
    dropout_l0: 0.2916944477688274
    n_units_l1: 290
    dropout_l1: 0.32461092127182756
    n_units_l2: 401
    dropout_l2: 0.45885496584447516
Loading model...
Test loss: 0.013417354518813747


# evaluate ans save on nwLH

### save txt file to make plot

In [9]:
Pk_type = 'MPk'
f_Pk      = 'all_MPk_nwLH.npy'        #file with Pk        #file with Pk
f_params  = '../real_params/nwLH_params.txt'       #file with parameters
mode = 'all'
extension = 'w_log_test_final_layer'
seed = 1
log = True
test_loader = data.create_dataset(mode, seed, f_Pk, None, f_params, 
                                  batch_size, shuffle=False, workers=1, cosm_type='BSQ', log=log)

cosm_type = 'nwLH'

# Store results
all_outputs = []
true_values = []

for x, y in test_loader:
    x = x.to(device)  # Move to device
    y = y.to(device)  # True cosmological parameters
    # outputs = model[:-2](x)  # Pass through the final hidden layer
    outputs = model(x)
    all_outputs.append(outputs.detach().cpu())  # Collect predictions
    true_values.append(y.detach().cpu())  # Collect true values

# Combine batches
all_outputs = torch.cat(all_outputs, dim=0)  # Predicted values: shape [N, 5]
true_values = torch.cat(true_values, dim=0)  # True values: shape [N, 5]

# Convert to NumPy
all_outputs_np = all_outputs.numpy()
true_values_np = true_values.numpy()

# Combine true and predicted values
results_np = np.hstack((true_values_np, all_outputs_np))  # Shape [N, 10]

# Save to file
np.savetxt('results/Results_'+mode+'_'+str(Pk_type)+'_'+str(cosm_type)+'_params_'+'_'+str(extension)+'_'+'fortransfer.txt', results_np, fmt='%.6e')

print(f"Saved results with shape: {results_np.shape}")



# reverse normalization
minimum = np.array([0.1, 0.02, 0.50, 0.80, 0.60])
maximum = np.array([0.5, 0.08, 0.90, 1.20, 1.00])
# new_Pk = all_outputs_np * (maximum - minimum) + minimum
new_params = true_values_np * (maximum - minimum) + minimum

# all_outputs_np = 10**all_outputs_np
np.save('all_'+str(Pk_type)+'_'+str(cosm_type)+'_'+str(extension)+'_fortransfer.npy', all_outputs_np)
# np.save()
# np.savetxt('../real_params/latin_hypercube_params_fixed_final_5_debug_2.txt', new_params)

Saved results with shape: (2000, 10)


In [8]:
data1 = np.load('all_Pk_nwLH_transfer_5.npy')
np.savetxt('data', data1)

In [3]:
model.load_state_dict

<bound method Module.load_state_dict of Sequential(
  (0): Linear(in_features=79, out_features=337, bias=True)
  (1): LeakyReLU(negative_slope=0.2)
  (2): Dropout(p=0.25292818538795103, inplace=False)
  (3): Linear(in_features=337, out_features=441, bias=True)
  (4): LeakyReLU(negative_slope=0.2)
  (5): Dropout(p=0.20604574607878604, inplace=False)
  (6): Linear(in_features=441, out_features=5, bias=True)
  (7): LeakyReLU(negative_slope=0.2)
  (8): Dropout(p=0.20370004600897423, inplace=False)
  (9): Linear(in_features=5, out_features=5, bias=True)
  (10): Sigmoid()
)>

In [8]:
model[:-2]

Sequential(
  (0): Linear(in_features=79, out_features=374, bias=True)
  (1): LeakyReLU(negative_slope=0.2)
  (2): Dropout(p=0.287554453774026, inplace=False)
  (3): Linear(in_features=374, out_features=210, bias=True)
  (4): LeakyReLU(negative_slope=0.2)
  (5): Dropout(p=0.2164435560349628, inplace=False)
  (6): Linear(in_features=210, out_features=5, bias=True)
  (7): LeakyReLU(negative_slope=0.2)
  (8): Dropout(p=0.20022563160490953, inplace=False)
)

In [67]:
help(model)

Help on Sequential in module torch.nn.modules.container object:

class Sequential(torch.nn.modules.module.Module)
 |  Sequential(*args)
 |  
 |  A sequential container.
 |  
 |  Modules will be added to it in the order they are passed in the
 |  constructor. Alternatively, an ``OrderedDict`` of modules can be
 |  passed in. The ``forward()`` method of ``Sequential`` accepts any
 |  input and forwards it to the first module it contains. It then
 |  "chains" outputs to inputs sequentially for each subsequent module,
 |  finally returning the output of the last module.
 |  
 |  The value a ``Sequential`` provides over manually calling a sequence
 |  of modules is that it allows treating the whole container as a
 |  single module, such that performing a transformation on the
 |  ``Sequential`` applies to each of the modules it stores (which are
 |  each a registered submodule of the ``Sequential``).
 |  
 |  What's the difference between a ``Sequential`` and a
 |  :class:`torch.nn.ModuleLi